In [1]:
import math
import os
import cv2 
import random
import numpy as np
from PIL import Image, ImageDraw

## Function to find boundingbox,center and area of objects

It finds top left and bottom right coordinates for bounding box around threat object and background object

It finds center of background object and area of threat object and background object

In [2]:
def boundingBox(img_name):
    
    img = cv2.imread(img_name,0)
    
    filter_img = cv2.GaussianBlur(img, (5, 5), 1.4) 
    _, thresh_img = cv2.threshold(filter_img, 245, 255, cv2.THRESH_BINARY_INV)
    
    contours, hierarchy = cv2.findContours(thresh_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_cnt = max(contours, key = cv2.contourArea)
    
    left, top, right, bottom = cv2.boundingRect(max_cnt)
    center = cv2.fitEllipse(max_cnt)
    area = cv2.contourArea(max_cnt)
 
    return left, top, right, bottom, center[0], area,
    

## Function to make transluscent mask for threat object

In [3]:
def translucent(img):
    
    numpy_img = np.array(img)  
    cv_img = cv2.cvtColor(numpy_img, cv2.COLOR_RGB2BGR) 
    gray_img = cv2.cvtColor(cv_img, cv2.COLOR_RGB2GRAY)
    
    filter_img = cv2.GaussianBlur(gray_img, (5, 5), 1.4)
    _, mask = cv2.threshold(filter_img, 245, 255, cv2.THRESH_BINARY_INV)
    
    mask[np.nonzero(mask)] = 150
    
    mask_arr = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
    mask_pil = Image.fromarray(mask_arr)
   
    return(mask_pil)

## Find and make list of Image files for threat and background 

In [4]:
cwd = os.getcwd()
    
for root, dirs, files in os.walk(".", topdown = False):

    for name in dirs:
        if name == "threat_images":
            threat_img_dir=os.path.join(root,name)
        if name == "background_images":
            bg_img_dir=os.path.join(root,name)
            
threat_img_list = os.listdir(threat_img_dir)
bg_img_list = os.listdir(bg_img_dir)

os.chdir(cwd)
if not os.path.exists('result_images'):
    os.makedirs('result_images')
os.chdir(cwd)

## Image processing


In [5]:
for j in range(0,len(bg_img_list)):
    
    bg_path = os.path.join(bg_img_dir,bg_img_list[j])
    
    _,_,_,_,C_bg,A_bg = boundingBox(bg_path)
    
    resize_l = int(math.sqrt(A_bg/8))
    img_bg = Image.open(bg_path)
    
    for i in range(0,len(threat_img_list)):

        img_path = os.path.join(threat_img_dir,threat_img_list[i])

        L_threat,T_threat,R_threat,B_threat,_,_ = boundingBox(img_path)
        
        img_threat = Image.open(img_path)
        
        random_x = random.randrange(int(C_bg[0]-resize_l/2),int(C_bg[0]+resize_l/4) , 1)
        random_y = random.randrange(int(C_bg[1]-resize_l/2),int(C_bg[1]+resize_l/4) , 1)

        crop_threat = img_threat.crop((L_threat,T_threat,L_threat+R_threat,T_threat+B_threat)) 
        rotate_threat = crop_threat.rotate(45,expand=True,fillcolor = (255,255,255))
        resize_threat = rotate_threat.resize((resize_l,resize_l))
        translucent_mask = translucent(resize_threat)
        
        img_output=img_bg.copy()
        
        img_output.paste(resize_threat,(random_x,random_y),translucent_mask.convert('L'))
        
        img_output.save(os.path.join('result_images','Output_bg'+str(j+1)+"_th"+str(i+1)+'.png'))
        
    